In [56]:
import json
from datetime import date, datetime, timedelta
import time
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import glob, os
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
import codecs

py.offline.init_notebook_mode(connected=True)

In [57]:
def parse_file(filename):
    
    file_dates = []
    file_msgs = []
    file_participants = []
    counter = 0
    with open(filename, encoding='latin2') as f:
        data = json.load(f)
        
        for participant in data['participants']:
            file_participants.append(participant['name'])
            file_msgs.append([])
            
        for message in data['messages']:
            photos = False
            cur_date = date.fromtimestamp(message['timestamp_ms'] / 1e3)

            if 'photos' in message:
                photos = True
            if cur_date in file_dates: #if date is already present!
                #print('date is present')
                if (photos):
                    for part in file_participants:
                        if (message['sender_name'] == part):
                            file_msgs[file_participants.index(message['sender_name'])][counter]+= len(message['photos'])
                        else:
                            file_msgs[file_participants.index(part)][counter]+=0
                else:
                    for part in file_participants:
                        if (message['sender_name'] == part):
                            file_msgs[file_participants.index(message['sender_name'])][counter]+= 1
                        else:
                            file_msgs[file_participants.index(part)][counter]+=0

            else: #if date is NOT present
                if(len(file_dates)!=0): #if dates is not empty (after the first loop)
                    #print('date is not present')
                    delta = file_dates[-1]-cur_date  #calculate difference between current date and last date in list
                
                    if (delta.days>0): #if there is a difference
                        for i in range(delta.days):                 #for the number of missing days do:
                            file_dates.append(file_dates[-1]-timedelta(days=1))      #add missing dates and add 0 to both msgs
                            counter+=1
                            #print('adding missing date')
                            if (file_dates[-1]==cur_date): #if last date is current date
                                #print('last date equals current')
                                if (photos):
                                    for part in file_participants:
                                        if (message['sender_name'] == part):
                                            file_msgs[file_participants.index(message['sender_name'])].append(len(message['photos']))
                                        else:
                                            file_msgs[file_participants.index(part)].append(0)
                                else:
                                    for part in file_participants:
                                        if (message['sender_name'] == part):
                                            file_msgs[file_participants.index(message['sender_name'])].append(1)
                                        else:
                                            file_msgs[file_participants.index(part)].append(0)
                            else:
                                for part in file_participants:
                                        file_msgs[file_participants.index(part)].append(0)
                
                else: # first loop (dates are empty)
                    #print('dates are empty')
                    file_dates.append(cur_date)
                    if (photos):
                        for part in file_participants:
                            if (message['sender_name'] == part):
                                file_msgs[file_participants.index(message['sender_name'])].append(len(message['photos']))
                            else:
                                file_msgs[file_participants.index(part)].append(0)
                    else:
                        for part in file_participants:
                            if (message['sender_name'] == part):
                                file_msgs[file_participants.index(message['sender_name'])].append(1)
                            else:
                                file_msgs[file_participants.index(part)].append(0)
                                                    
    return file_dates,file_msgs,file_participants

In [58]:
dates = []
msgs = []
counter = 0
participants = []
msg_counter = 0
os.chdir("C:\\Users\\Qbook\\fejsik\\messages\\inbox\\JKStillSmellsLIKEGwynethsVag_0wV4y5omMA") #insert folder dir

for file in glob.glob("*.json"):
    #print(file)
    
    
    filename = os.getcwd()+"//"+file
    cur_dates, cur_msgs, participants = parse_file(filename)

    if(len(dates)!=0): #if data not empty 
        #print('data not empty')
        delta = dates[-1]-cur_dates[0]
        temp_dates = dates
        #temp_msgs_in = msgs_in
        #temp_msgs_out = msgs_out
        
    
        if(delta.days>1):
            #print('missing days')
            for i in range(delta.days):
                temp_dates.append(temp_dates[-1]+timedelta(days=1))
                for part in participants:
                    msgs[participants.index(part)].append(0)
                #temp_msgs_in.append(0)
                #temp_msgs_out.append(0)
        else:
            #print('overlapping days')
            if(delta.days==0):
                #temp_overlapping_msgs= []
                #temp_in = cur_msgs_in[-1]
                #temp_out = cur_msgs_out[-1]
                #for part in participants:
                #    temp_overlapping_msgs.append(cur_msgs[participants.index(part)][-1])
                
                #temp_msgs_in[0]+=temp_in
                #temp_msgs_out[0]+=temp_out
                for part in participants:
                    msgs[participants.index(part)][0]+=cur_msgs[participants.index(part)][-1]
                del cur_dates[-1]
                #del cur_msgs_in[-1]
                #del cur_msgs_out[-1]
                for part in participants:
                    del cur_msgs[participants.index(part)][-1]
            
        #msgs_in=temp_msgs_in+cur_msgs_in
        #msgs_out=temp_msgs_out+cur_msgs_out
        for part in participants:  #check if does work! (if you can add lists with +=)
            msgs[participants.index(part)]+=cur_msgs[participants.index(part)]
        dates=temp_dates+cur_dates
    else:
        #print('data empty')
        #msgs_in = cur_msgs_in
        #msgs_out = cur_msgs_out
        #print(msgs)
        #print(participants.index(part))
        #print(cur_msgs[participants.index(part)])
        #print(msgs[participants.index(part)])
        #for part in participants:
        #    msgs[participants.index(part)] = cur_msgs[participants.index(part)]
        msgs = cur_msgs
        dates = cur_dates
    #print(dates[0])
    #print(dates[-1])     
    print('----')
    #print(temp_msgs_in)
    #print(temp_msgs_out)
    #print(temp_dates)
print(dates[0])
print(dates[-1])
#print(msgs)

----
----
2020-01-15
2019-03-07


In [59]:
print((dates[0]-dates[-1]).days)
print(len(msgs[0]))
print(len(msgs[1]))
print(len(dates))

314
316
316
316


In [60]:
#df=pd.DataFrame({'x':dates, 'y1':msgs_in, 'y2':msgs_out})
#df_sns=pd.DataFrame({msgs_in, msgs_out, dates})

In [61]:
#plt.figure(figsize=(30, 30))
#plt.plot('x','y1', data=df, color='blue')
#plt.plot('x','y2', data=df, color='green')

In [62]:
layout = go.Layout(
title='example',
    yaxis=dict(
        title='number of messages'
    ),
    xaxis=dict(
        title='date'
    )
)

traces = []
#for part in participants:
#    part=part.encode('utf-8')

for idx, participant in enumerate(msgs):
    trace = go.Scatter(
        x=dates,
        y=participant,
        mode='lines',
        name=participants[idx],
        line=dict(
        shape='spline'
        )
    )
    traces.append(trace)

fig = go.Figure(traces, layout=layout)

py.offline.iplot(fig)
